In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import requests
from bs4 import BeautifulSoup
import datetime
import os
import shutil

if not os.path.exists("graphic/temporalSeries"):
    os.makedirs("graphic/temporalSeries")
else:
    shutil.rmtree("graphic/temporalSeries")
    os.makedirs("graphic/temporalSeries")

if not os.path.exists("models/logs/temporalSeries"):
    os.makedirs("models/logs/temporalSeries")
if not os.path.exists("prediction/logs/temporalSeries"):
    os.makedirs("prediction/logs/temporalSeries")

In [2]:
#Introducción
print("***************Introducción*******************")
print("Se intenta ampliar la capacidad de predicción con el uso de series temporales y no solo con una regresión linear")

***************Introducción*******************
Se intenta ampliar la capacidad de predicción con el uso de series temporales y no solo con una regresión linear


In [3]:
# Fetch the list of tickers from the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response = requests.get(url,verify=False)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find("table", {"class": "wikitable sortable"})
tickers = []
for row in table.findAll("tr")[1:]:
    ticker = row.findAll("td")[0].text.strip()
    tickers.append(ticker)

print(tickers)

C:\Users\aitor\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'D

In [ ]:
# Fetch historical data for each ticker
for ticker in tickers:
    nowDate = datetime.datetime.now()
    try:
        try:
            if "." in ticker:          #Yahoo Finance uses - instead of . (ex. BRK.B -> BRK-B)
                ticker = ticker.split(".")
                ticker = ticker[0]+ "-" + ticker[1]
            data = yf.download(ticker)

        except ValueError as e:
            print(f"Couldn't fetch data for {ticker}")
            with open(f"models/logs/{ticker}.txt", "a") as f:
                    f.write(f"[{nowDate}]{ticker} : {e}\n")
            continue
        
        # Rename columns to match Prophet's expectations
        stock_data = data.reset_index()
        stock_data = stock_data.rename(columns={'Close': 'y','Date': 'ds'})
        # Create the Prophet model
        model = Prophet(daily_seasonality=True)

        # Fit the model on the stock data
        model.fit(stock_data)
        print(f'Model for {ticker} has been trained')

        # Make predictions for the next 10 days
        future_dates = model.make_future_dataframe(periods=10)
        forecast = model.predict(future_dates)

        # Plot the forecast
        model.plot(forecast)

        plt.xlabel('Year')
        plt.ylabel(f'Predicted Stock Price {ticker}')

        # Save the plot to an archive
        plt.savefig(f"graphic/temporalSeries/{ticker}.png")
                
    except ValueError as e:
        print(f"Couldn't fetch data for {ticker}")
        with open(f"models/logs/{ticker}.txt", "a") as f:
                f.write(f"[{nowDate}]{ticker} : {e}\n")    

[*********************100%***********************]  1 of 1 completed


C:\Users\aitor\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Model for MMM has been trained


C:\Users\aitor\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
C:\Users\aitor\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


[*********************100%***********************]  1 of 1 completed
Model for AOS has been trained


...

C:\Users\aitor\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


C:\Users\aitor\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
C:\Users\aitor\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


[*********************100%***********************]  1 of 1 completed
Model for FLT has been trained


C:\Users\aitor\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
C:\Users\aitor\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
C:\Users\aitor\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


[*********************100%***********************]  1 of 1 completed

In [1]:
#Resultados
print("***************Resultados*******************")
print("Hemos conseguido mostrar un rango de predicción de los valores con un campo en el que puede moverse el precio, basado en datos históricos")
print("Se ha conseguido poder guardar ese rango de forma visual ese rango para el futuro y para todas las acciones del sp500")
print("Proveemos de forma visual de un posible rango de precios lógicos para la accion basado en su volatilidad histórica que puede ayudar a un inversor a ver si una corrección es irracional o no")

***************Resultados*******************
Hemos conseguido mostrar un rango de predicción de los valores con un campo en el que puede moverse el precio, basado en datos históricos
Se ha conseguido poder guardar ese rango de forma visual ese rango para el futuro y para todas las acciones del sp500
Proveemos de forma visual de un posible rango de precios lógicos para la accion basado en su volatilidad histórica que puede ayudar a un inversor a ver si una corrección es irracional o no


In [2]:
#Mejoras
print("***************Mejoras propuestas*******************")
print("Obviamente el precio que obtenemos no tiene en cuenta eventos como presentacion de resultados, dividendos, etc. Sería un buen punto añadido")
print("Actualmente al descargar los datos de manera dinámica se requiere de acceso a internet, y la api de Yahoo finance puede estar caida, lo que desemboca problemas")
print("Dar soporte a mas ínidces bursátiles y acciones en particular")

***************Mejoras propuestas*******************
Obviamente el precio que obtenemos no tiene en cuenta eventos como presentacion de resultados, dividendos, etc. Sería un buen punto añadido
Actualmente al descargar los datos de manera dinámica se requiere de acceso a internet, y la api de Yahoo finance puede estar caida, lo que desemboca problemas
Dar soporte a mas ínidces bursátiles y acciones en particular
